## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append("../")
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,60.91,82,0,11.99,clear sky
1,1,Butaritari,KI,3.0707,172.7902,81.86,72,18,12.17,few clouds
2,2,Mataura,NZ,-46.1927,168.8643,61.48,66,53,2.66,broken clouds
3,3,Homer,US,59.6425,-151.5483,40.89,70,100,3.44,light rain
4,4,Busselton,AU,-33.6500,115.3333,66.34,58,96,5.84,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                196
City                   196
Country                196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                196
City                   196
Country                196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.86,few clouds,3.0707,172.7902,
5,Rikitea,PF,78.40,clear sky,-23.1203,-134.9692,
14,Faanui,PF,80.89,light rain,-16.4833,-151.7500,
17,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
31,Hithadhoo,MV,83.71,scattered clouds,-0.6000,73.0833,
33,Atuona,PF,79.41,light rain,-9.8000,-139.0333,
35,Maumere,ID,85.14,overcast clouds,-8.6199,122.2111,
39,Manggar,ID,86.23,scattered clouds,-2.8833,108.2667,
49,Saint-Philippe,RE,79.21,scattered clouds,-21.3585,55.7679,
52,Kavieng,PG,85.77,scattered clouds,-2.5744,150.7967,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,81.86,few clouds,3.0707,172.7902,Isles Sunset Lodge
5,Rikitea,PF,78.40,clear sky,-23.1203,-134.9692,People ThankYou
14,Faanui,PF,80.89,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
31,Hithadhoo,MV,83.71,scattered clouds,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...,...,...
657,Pilar,BR,75.13,overcast clouds,-9.5972,-35.9567,Pousada e churrascaria Manguaba
658,Dwarka,IN,83.86,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
659,Anloga,GH,81.97,light rain,5.7947,0.8973,Pin Drop Hotel
668,Lasem,ID,87.96,scattered clouds,-6.6922,111.4527,"Nyah Lasem | Museum, Warung & Community Center"


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))